# String Method Analysis Markov-State-Models
## Imports

In [ ]:
%load_ext lab_black

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
import os
import pickle
import sys
import logging
import numpy as np
import pyemma

logging.getLogger("matplotlib").setLevel(logging.ERROR)
logging.getLogger("blib2to3.pgen2.driver").setLevel(logging.WARNING)
logging.getLogger("pyemma").setLevel(logging.NOTSET)
sys.path.append("../string-method-gmxapi/")
from stringmethod.config import *
from stringmethod.postprocessing import *

In [ ]:
def colorbar(mappable, cmap, norm, label0, size=10):
    from mpl_toolkits.axes_grid1 import make_axes_locatable

    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, norm=norm)
    cbar.set_label(label0, size=size)
    return cbar

In [ ]:
def load_swarm_data(extract, first_iteration=1, last_iteration=None):
    if last_iteration == None:
        last_iteration = sys.maxsize
    if extract:
        config = load_config("config.json")

        ce = CvValueExtractor.from_config(
            config=config,
            first_iteration=first_iteration,  # Exclude the first iterations to let the system equilibrate.
            last_iteration=last_iteration,  # Usefull to make blocks of the simulation
        )
        ce.run()
        ce.persist()
    return np.load("postprocessing/cv_coordinates.npy")

In [ ]:
plt.rcParams["axes.facecolor"] = "#f9f9fb"
plt.rcParams["grid.color"] = "white"
plt.rcParams["grid.linestyle"] = "-"
plt.rcParams["grid.linewidth"] = 2
plt.rcParams["axes.grid"] = True
plt.rcParams["lines.solid_capstyle"] = "round"

## Load data

In [ ]:
%ls ../data/raw

In [ ]:
simulation_directory = "/data/sperez/Projects/string_sims/data/raw/C2I_lb_v2/"
os.chdir(simulation_directory)
os.getcwd()

In [ ]:
with open("cv.pkl", "rb") as file:
    cvs, ndx_groups = pickle.load(file)

The `load_swarm_data` function will load the swarm data in the `cv_coordinates`. If you set `extract=True` it will read the data from the swarm files. If you have done this previously you can set `extract=False` so the function just reads `postprocessing/cv_coordinates.npy`. `first_iteration` can be used to exclude initial swarms as equilibration and `last_iteration` can be done to exclude some iterations for example if you want to estimate the FES convergence by comparing blocks of data.

In [ ]:
cv_coordinates0 = load_swarm_data(
    extract=False, first_iteration=100, last_iteration=None
)

## Choose cvs to use

### CVs for MSMs

In [ ]:
my_cvs = [0, 6]
cv_coordinates = cv_coordinates0[:, :, my_cvs]

In [ ]:
cv_coordinates = cv_coordinates0

In [ ]:
cv_coordinates = np.concatenate(
    [
        np.concatenate(
            [cv_coordinates0[:, :, 0:1], cv_coordinates0[:, :, 1:2]], axis=0
        ),
        np.concatenate(
            [cv_coordinates0[:, :, 11:12], cv_coordinates0[:, :, 11:12]], axis=0
        ),
    ],
    axis=2,
)

### CVs for projection

In [ ]:
my_cvs = [0, 6]
cv_proj = cv_coordinates0[:, :, my_cvs]

In [ ]:
cv_proj = np.concatenate(
    [
        np.concatenate(
            [cv_coordinates0[:, :, 0:1], cv_coordinates0[:, :, 1:2]], axis=0
        ),
        np.concatenate(
            [cv_coordinates0[:, :, 10:11], cv_coordinates0[:, :, 11:12]], axis=0
        ),
    ],
    axis=2,
)

In [ ]:
cv_proj = cv_coordinates

### Prepare data for pyemma

In [ ]:
data = []
for i in range(cv_coordinates.shape[0]):
    data.append(cv_coordinates[i, :, :])

In [ ]:
data_proj = []
for i in range(cv_proj.shape[0]):
    data_proj.append(cv_proj[i, :, :])

In [ ]:
data_concat = np.concatenate(data)

In [ ]:
data_proj_concat = np.concatenate(data_proj)

## Cluster

In [ ]:
def score_cv(data, dim, lag, number_of_splits=10, validation_fraction=0.5):
    """Compute a cross-validated VAMP2 score.

    We randomly split the list of independent trajectories into
    a training and a validation set, compute the VAMP2 score,
    and repeat this process several times.

    Parameters
    ----------
    data : list of numpy.ndarrays
        The input data.
    dim : int
        Number of processes to score; equivalent to the dimension
        after projecting the data with VAMP2.
    lag : int
        Lag time for the VAMP2 scoring.
    number_of_splits : int, optional, default=10
        How often do we repeat the splitting and score calculation.
    validation_fraction : int, optional, default=0.5
        Fraction of trajectories which should go into the validation
        set during a split.
    """
    # we temporarily suppress very short-lived progress bars
    pyemma.util.config.show_progress_bars = False
    nval = int(len(data) * validation_fraction)
    scores = np.zeros(number_of_splits)
    for n in range(number_of_splits):
        ival = np.random.choice(len(data), size=nval, replace=False)
        vamp = pyemma.coordinates.vamp(
            [d for i, d in enumerate(data) if i not in ival], lag=lag, dim=dim
        )
        scores[n] = vamp.score([d for i, d in enumerate(data) if i in ival])
    pyemma.util.config.show_progress_bars = True
    return scores

In [ ]:
n_clustercenters = [5, 10, 30, 75, 200, 450, 650]
scores = np.zeros((len(n_clustercenters), 5))
pyemma.util.config.show_progress_bars = True
clusters = []
for n, k in enumerate(n_clustercenters):
    print(k)
    for m in range(5):
        _cl = pyemma.coordinates.cluster_kmeans(data, k=k, max_iter=50, stride=1)
        _msm = pyemma.msm.estimate_markov_model(_cl.dtrajs, 1)
        scores[n, m] = _msm.score_cv(
            _cl.dtrajs, n=1, score_method="VAMP2", score_k=min(10, k)
        )
    clusters.append(_cl)
fig, ax = plt.subplots()
lower, upper = pyemma.util.statistics.confidence_interval(scores.T.tolist(), conf=0.9)
ax.fill_between(n_clustercenters, lower, upper, alpha=0.3)
ax.plot(n_clustercenters, np.mean(scores, axis=1), "-o")
ax.semilogx()
ax.set_xlabel("number of cluster centers")
ax.set_ylabel("VAMP-2 score")
fig.tight_layout()

In [ ]:
def get_cluster_plot(data, data_proj_concat, k, ax):
    cluster = pyemma.coordinates.cluster_kmeans(
        data, k=k, max_iter=200, stride=1, fixed_seed=1
    )
    dtrajs_concatenated = np.concatenate(cluster.dtrajs)
    pyemma.plots.plot_density(*data_proj_concat.T, ax=ax, cbar=False, alpha=0.3)
    ax.scatter(*cluster.clustercenters.T, s=5, c="C1")
    ax.set_xlabel("CV 0")
    ax.set_ylabel("CV 1")
    return ax, cluster

In [ ]:
k = 75
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
ax, cluster = get_cluster_plot(data, data_proj_concat, k, ax)
fig.tight_layout()

## MSM

In [ ]:
msm = pyemma.msm.estimate_markov_model(
    cluster.dtrajs,
    lag=1,
    dt_traj="0.01 ns",
    reversible=True,
)
print("fraction of states used = {:.2f}".format(msm.active_state_fraction))
print("fraction of counts used = {:.2f}".format(msm.active_count_fraction))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharex=True, sharey=True)
# av_y = np.mean(reduced_string[start_string:, 0, :], axis=0)
# std_y = np.std(reduced_string[start_string:, 0, :], axis=0)
# av_x = np.mean(reduced_string[start_string:, 1, :], axis=0)
# std_x = np.std(reduced_string[start_string:, 1, :], axis=0)
# axes[0].fill_between(
#     av_x,
#     av_y + std_y,
#     av_y - std_y,
#     alpha=0.1,
#     label=f"std_x(string{start_string}-{n_strings})",
#     color="cyan",
# )
# axes[0].fill_betweenx(
#     av_y,
#     av_x + std_x,
#     av_x - std_x,
#     alpha=0.1,
#     label=f"std_y(string{start_string}-{n_strings})",
#     color="cyan",
# )
# axes[0].plot(
#     av_x,
#     av_y,
#     ls="-",
#     marker=".",
#     color="k",
#     label=f"mean(string{start_string}-{n_strings})",
# )
# axes[1].fill_between(
#     av_x,
#     av_y + std_y,
#     av_y - std_y,
#     alpha=0.1,
#     label=f"std_x(string{start_string}-{n_strings})",
#     color="cyan",
# )
# axes[1].fill_betweenx(
#     av_y,
#     av_x + std_x,
#     av_x - std_x,
#     alpha=0.1,
#     label=f"std_y(string{start_string}-{n_strings})",
#     color="cyan",
# )
# axes[1].plot(
#     av_x,
#     av_y,
#     ls="-",
#     marker=".",
#     color="k",
#     label=f"mean(string{start_string}-{n_strings})",
# )
pyemma.plots.plot_free_energy(
    *data_proj_concat[:, ::-1].T, ax=axes[0], mask=True, cbar_label="non_reweighted"
)
pyemma.plots.plot_free_energy(
    *data_proj_concat[:, ::-1].T,
    weights=np.concatenate(msm.trajectory_weights()),
    ax=axes[1],
    legacy=False,
    cmap="RdYlBu_r",
    # vmax=12,
)
for ax in axes.flat:
    ax.set_xlabel("IC 1")
axes[0].set_ylabel("IC 2")
axes[0].set_title("non_reweighted", fontweight="bold")
axes[1].set_title("Reweighted free energy surface", fontweight="bold")
fig.tight_layout()